1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [11]:
# input number and its base
def change_rep(n, base):
    if base == 2:
        n_bin = n
        n_dec = int(n, 2)
        n_hex = hex(n_dec)
    elif base == 10:
        n_bin = bin(n)
        n_dec = n
        n_hex = hex(n)
    elif base == 16:
        n_dec = int(n, 16)
        n_bin = bin(n_dec)
        n_hex = hex(n_dec)
    else:
        print("Aborting: invalid base.")
        return None
    print("n_2 =", n_bin, "\nn_10 =", n_dec, "\nn_16 =", n_hex)
    
# example
n = "1100"
change_rep(n, 2)
print("---")
n = "fb"
change_rep(n, 16)
print("---")
n = 15
change_rep(n, 10)

n_2 = 1100 
n_10 = 12 
n_16 = 0xc
---
n_2 = 0b11111011 
n_10 = 251 
n_16 = 0xfb
---
n_2 = 0b1111 
n_10 = 15 
n_16 = 0xf


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [13]:
def bfloat(n):
    if len(n) != 32:
        print("Aborting: not a 32 bit string")
        return None
    
    sign = (-1)**int(n[0], 2)
    exponent = int(n[1:9], 2)
    m = n[9:]
    f = 0
    for i in range(0, len(m)):
        f += int(m[i]) * 2**(-i-1)
    mantissa = 1 + f
    res = sign * 2**(exponent-127) * mantissa
    
    return res
     
# example  
n = "11000000101100000000000000000000"
bfloat(n)

-5.5

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [18]:
a = 1
b = 1
notFound = True

# determine underflow
while notFound:
    c = a / 2
    if c > 0:
        a /= 2
    else:
        notFound = False
        print("Underflow =", a)

# determine overflow
notFound = True
while notFound:
    d = b * 2
    if d != b:
        b *= 2
    else:
        notFound = False
        print("Overflow =", b)
        
'''
TO DO: overflow
'''


Underflow = 5e-324


KeyboardInterrupt: 

In [19]:
# check information about float in my system
import sys
print(sys.float_info)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [1]:
notFound = True
before, after, i = 0, 0, 0

while notFound:
    after += 10**(-i)
    if before < after:
        before = after
        i += 1
    else:
        notFound = False
        minPower = i

print("Minimum power reached =", -minPower)

Minimum power reached = -16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [23]:
from math import sqrt

In [25]:
# solution (a)
def findSol_1(a, b, c):
    val = sqrt( b**2 - 4*a*c )
    return( (- b + val) / (2 * a), (- b - val) / (2 * a) )

# example
findSol_1(0.001, 1000, 0.001)

(-9.999894245993346e-07, -999999.999999)

In [28]:
# solution (b)
def findSol_2(a, b, c):
    val = sqrt( b**2 - 4*a*c )
    return( (2 * c) / (- b + val), (2 * c) / (- b - val) )

# example
findSol_2(0.001, 1000, 0.001)

'''
    TO DO: COMMENT ON THE OUTPUT (roundoff error)
'''

'\n    TO DO: COMMENT ON THE OUTPUT (roundoff error)\n'

In [29]:
# solution (c)
'''
    TO DO: usare la prima formula per una soluzione e l'altra formula per l'altra
'''

"\n    TO DO: usare la prima formula per una soluzione e l'altra formula per l'altra\n"

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [33]:
# solution (a)
def f(x):
    return x**2 - x

def f_p(x):
    return 2*x - 1

def df(x, d):
    return (f(x+d) - f(x)) / d

print("True value =", f_p(1))
print("Using the definition =", df(1, 10**(-2)))

'''
    TO DO: COMMENT (sovrastima perche e grande delta)
'''

True value = 1
Using the definition = 1.0099999999999998


'\n    TO DO: COMMENT (sovrastima perche e grande delta)\n'

In [36]:
# solution (b)
dl = [10**(-c) for c in [4, 6, 8, 10, 12, 14]]

for val in dl:
    print("Using d =", val, "the derivative is", df(1, val))

'''
    TO DO: COMMENT
'''

Using d = 0.0001 the derivative is 1.0000999999992821
Using d = 1e-06 the derivative is 1.000001000006634
Using d = 1e-08 the derivative is 0.999999993922529
Using d = 1e-10 the derivative is 1.000000082740371
Using d = 1e-12 the derivative is 1.000088900582341
Using d = 1e-14 the derivative is 0.9992007221626409


'\n    TO DO: COMMENT\n'

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [53]:
# solution (a)
def g(x):
    return sqrt(1 - x**2)

# even function, calculate integral in [0, 1] 
# and finally multiply by 2
def int_g(N):
    h = 1 / N
    int = 0
    for k in range(0, N):
        int += h * g(h*k)
    return 2*int
        
int_g(100)

'''
    TO DO: sistema tutto e finisci.
'''

1.5802085158895236